In [59]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML, display
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [60]:
# Load useful libraries
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.metrics import *

# Plotting libraries
import matplotlib.pyplot as plt
plt.gray()

<Figure size 432x288 with 0 Axes>

In [61]:
# Run this cell
# Assumes CelebA has been manually downloaded and is available in `~/tensorflow_datasets/celeb_a/2.0.0/`.

import tensorflow_datasets as tfds

train_celeb, val_celeb = tfds.load('celeb_a', split=['train', 'validation'], shuffle_files=False, download=False)

In [62]:
# Global variables to define training/loading models. 
# Modify as required. These are only suggested parameters.

train = True
epochs = 5  # number of epochs to train models
batch_size = 32
input_size = (64, 64, 3)  # images will be cropped and resized to `input_size`.

In [63]:
# DO NOT DELETE THIS CELL
# Use this function to normalize, crop and resize your images.
def tf_norm_crop_resize_image(image, resize_dim):
    """Normalizes image to [0.,1.], crops to dims (150, 150, 3)
    and resizes to `resize_dim`, returning an image tensor."""
    image = tf.cast(image, tf.float32)/255.
    image = tf.image.resize_with_crop_or_pad(image, 150, 150)
    image = tf.image.resize(image, resize_dim)
    image.set_shape(resize_dim + (3,))
    return image

In [64]:
def tf_resize_helper(element):
    image = element['image']
    image = tf_norm_crop_resize_image(image,(64,64))
    return (image,image)

In [65]:
seed = 30
prefetch = 500
train_ds = train_celeb.map(tf_resize_helper).shuffle(1000, seed = 30).batch(batch_size).prefetch(prefetch)
val_ds = train_celeb.map(tf_resize_helper).shuffle(1000, seed = 30).batch(batch_size).prefetch(prefetch)

In [66]:
features = next(iter(train_ds.take(1)))
features[0].shape, features[1].shape

(TensorShape([32, 64, 64, 3]), TensorShape([32, 64, 64, 3]))

In [80]:
input_size, input_img

((64, 64, 3), <tf.Tensor 'input_6:0' shape=(None, 64, 64, 3) dtype=float32>)

In [79]:
latent_dim = 11
dropout_rate = 0.0
input_img = tf.keras.layers.Input(shape = input_size)

x = tf.keras.layers.Conv2D(64,(5,5), padding = 'same', strides = (2,2), activation = 'relu')(input_img)
x = tf.keras.layers.Dropout(dropout_rate)(x)
x = tf.keras.layers.Conv2D(32,(5,5), padding = 'same', strides = (2,2), activation = 'relu')(x)
x = tf.keras.layers.Dropout(dropout_rate)(x)
x = tf.keras.layers.Conv2D(18,(5,5), padding = 'same', strides = (2,2), activation = 'relu')(x)
x = tf.keras.layers.Dropout(dropout_rate)(x)
x = tf.keras.layers.Conv2D(9,(5,5), padding = 'same', strides = (2,2), activation = 'relu')(x)
x = tf.keras.layers.Dropout(dropout_rate)(x)

shape_before_flattening = K.int_shape(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation = 'linear')(x)
x = tf.keras.layers.Dense(2, activation = 'relu')(x)

# latent mean and log variance
z_mu = tf.keras.layers.Dense(latent_dim)(x)
z_log_sigma = tf.keras.layers.Dense(latent_dim)(x)

In [68]:
# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim), mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = tf.keras.layers.Lambda(sampling)([z_mu, z_log_sigma])

In [69]:
# instantiate encoder model
encoder = tf.keras.models.Model(input_img, [z_mu, z_log_sigma, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 32, 32, 64)   4864        input_4[0][0]                    
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 32, 32, 64)   0           conv2d_4[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 16, 16, 32)   51232       dropout_10[0][0]                 
____________________________________________________________________________________________

In [70]:
# decoder

# latent distribution shape
decoder_input = tf.keras.layers.Input(K.int_shape(z)[1:])
x = tf.keras.layers.Dense((2), activation = 'relu')(decoder_input)
x = tf.keras.layers.Dense((128), activation = 'relu')(x)# (decoder_input)#(x)

x = tf.keras.layers.Dense(np.prod(shape_before_flattening[1:]), activation = 'relu')(x) #(decoder_input)

x = tf.keras.layers.Reshape(shape_before_flattening[1:])(x)

#x = tf.keras.layers.Conv2DTranspose(9,(5,5), padding = 'same', strides = (2,2), activation = 'relu')(x)
#x = tf.keras.layers.Dropout(dropout_rate)(x)
x = tf.keras.layers.Conv2DTranspose(18,(5,5), padding = 'same', strides = (2,2), activation = 'relu')(x)
x = tf.keras.layers.Dropout(dropout_rate)(x)
x = tf.keras.layers.Conv2DTranspose(32,(5,5), padding = 'same', strides = (2,2), activation = 'relu')(x)
x = tf.keras.layers.Dropout(dropout_rate)(x)
x = tf.keras.layers.Conv2DTranspose(64,(5,5), padding = 'same', strides = (2,2), activation = 'relu')(x)
x = tf.keras.layers.Dropout(dropout_rate)(x)
x = tf.keras.layers.Conv2DTranspose(3,(5,5), padding = 'same',activation = 'relu', strides = (2,2))(x)

decoder = tf.keras.Model(decoder_input,x)
z_decoded = decoder(z)
decoder.summary()


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 24        
_________________________________________________________________
dense_15 (Dense)             (None, 128)               384       
_________________________________________________________________
dense_16 (Dense)             (None, 144)               18576     
_________________________________________________________________
reshape_2 (Reshape)          (None, 4, 4, 9)           0         
_________________________________________________________________
conv2d_transpose_8 (Conv2DTr (None, 8, 8, 18)          4068      
_________________________________________________________________
dropout_14 (Dropout)         (None, 8, 8, 18)          0   

In [71]:
# construct a custom layer to calculate the loss
class CustomVariationalLayer(layers.Layer):

    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)

        # Reconstruction loss
        xent_loss = tf.keras.losses.binary_crossentropy(x, z_decoded)
        
        # KL divergence
        kl_loss = -5e-4 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
        return K.mean(xent_loss + kl_loss)

    # adds the custom loss to the class
    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

# apply the custom loss to the input images and the decoded latent distribution sample
y = CustomVariationalLayer()([input_img, z_decoded])

In [73]:
vae = tf.keras.Model(input_img, y)
vae.compile(optimizer = 'rmsprop', loss = 'mse')

In [74]:
vae.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 32, 32, 64)   4864        input_4[0][0]                    
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 32, 32, 64)   0           conv2d_4[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 16, 16, 32)   51232       dropout_10[0][0]                 
____________________________________________________________________________________________

In [75]:
# Global variables to define training/loading models. 
# Modify as required. These are only suggested parameters.

train = True
epochs = 5  # number of epochs to train models
batch_size = 32
input_size = (64, 64, 3)  # images will be cropped and resized to `input_size`.

In [76]:
type(train_ds), type(train_celeb)

(tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter,
 tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter)

In [81]:
for elem in train_ds:
    train_ds_x = 

TypeError: 'DatasetV1Adapter' object is not subscriptable

In [ ]:
mnist_train, mnist_test = mnist_data["train"], mnist_data["test"]

train_ds_x, train_ds_y = train_ds['image'],

In [77]:
%%time

# callbacks = [ 
#             tf.keras.callbacks.EarlyStopping(
#             # Stop training when `val_accuracy` is no longer improving
#             monitor='val_accuracy',
#             # "no longer improving" being further defined as "for at least 2 epochs"
#             patience=2,
#             verbose=1)
#             ]

history = vae.fit(train_ds.batch(batch_size), epochs = epochs, validation_data = val_ds.batch(batch_size))


Epoch 1/5
      1/Unknown - 0s 126ms/step

ValueError: Error when checking input: expected input_4 to have 4 dimensions, but got array with shape (None, None, 64, 64, 3)